# Exploration des aides des l'ADEME.

Nous explorons des aides de l'ADEME, en open data :
https://data.ademe.fr/datasets/les-aides-financieres-de-l'ademe

Le but ici pour notre projet est de récupérer des couples aides/entreprises pour tenter d'automatiser cette association. Pour ramener ce matching dans le cadre de mission transition il faudrait pouvoir retrouver les aides de mission transition publiées, originaires de l'ADEME dans les donnée de l'ADEME. Ainsi nous obtiendrons des aides de mission transition enrichies des entreprises qui ont obtenue une subvention pour ces aides. Grâce à l'API SIRENE notament nous obtiendrons alors de nombreuses inforamtions pour tenter le matching.

## Récupération des aides de l'ADEME.

In [1]:
import os
import pandas as pd
import requests
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
# File path
DATA_FOLDER = "../data"
DATA_FILE = "Les aides financières ADEME-full.csv"
path_to_data = os.path.join(DATA_FOLDER, DATA_FILE)

# import into a Dataframe
df_ademe = pd.read_csv(path_to_data)
df_ademe

,Nom de l attribuant,idAttribuant,dateConvention,referenceDecision,nomBeneficiaire,idBeneficiaire,objet,montant,nature,conditionsVersement,...,notificationUE,pourcentageSubvention,location.lat,location.lon,DEPET,naf1etlib,naf2etlib,naf3etlib,naf4etlib,naf5etlib
0,ADEME,38529030900454,2021-04-12,21BFD0081,CLEMENT SAS,7.269205e+13,TREMPLIN pour la transition écologique des PME,6500.00,aide en numéraire,Echelonné,...,NaN,1.00,46.671862,4.365152,71,Industrie manufacturière,Fabrication de machines et équipements n.c.a.,Fabrication de machines de formage des métaux ...,Fabrication de machines de formage des métaux,Fabrication de machines-outils pour le travail...
1,ADEME,38529030900454,2021-03-25,21BFD0119,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,3.950618e+13,TREMPLIN pour la transition écologique des PME,12150.00,aide en numéraire,Echelonné,...,NaN,1.00,47.105037,5.504615,39,Industrie manufacturière,"Fabrication de produits métalliques, à l'excep...",Traitement et revêtement des métaux ; usinage,Usinage,Mécanique industrielle
2,ADEME,38529030900454,2021-03-23,21BFD0123,TRANSDEV BFC SUD,3.391339e+13,TREMPLIN pour la transition écologique des PME,16000.00,aide en numéraire,Echelonné,...,NaN,1.00,46.790418,4.833945,71,Transports et entreposage,Transports terrestres et transport par conduites,Autres transports terrestres de voyageurs,Autres transports terrestres de voyageurs n.c.a.,Transports routiers réguliers de voyageurs
3,ADEME,38529030900454,2021-03-23,21BFD0137,LOCA TRAVAUX EURL,4.999836e+13,TREMPLIN pour la transition écologique des PME,16400.00,aide en numéraire,Echelonné,...,NaN,1.00,47.623341,6.957088,90,Activités de services administratifs et de sou...,Activités de location et location-bail,"Location et location-bail d'autres machines, é...",Location et location-bail de machines et équip...,Location et location-bail de machines et équip...
4,ADEME,38529030900454,2021-03-26,21BFD0142,RODESCHINI SAS,3.008465e+13,TREMPLIN pour la transition écologique des PME,21500.00,aide en numéraire,Echelonné,...,NaN,1.00,NaN,NaN,70,Construction,Travaux de construction spécialisés,Autres travaux de construction spécialisés,Autres travaux de construction spécialisés n.c.a.,Travaux de maçonnerie générale et gros œuvre d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12691,ADEME,38529030900454,2019-07-29,1905C0008,ASSOCIATION FRANCAISE DES PROFESSIONNELS DE L...,5.252716e+13,Soutien au programme d'actions 2019 de l'AFPG,88208.94,aide en numéraire,Echelonné,...,1.0,0.21,48.839829,2.346895,75,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil en relations publiques et communication,Conseil en relations publiques et communication
12692,ADEME,38529030900454,2019-07-12,1906C0002,MAITRES OUVRAGES PUBLICS,3.477621e+13,"Soutien à la réalisation du plan d'actions ""dé...",131930.69,aide en numéraire,Echelonné,...,1.0,0.20,45.771153,4.864377,69,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil pour les affaires et autres conseils d...,Conseil pour les affaires et autres conseils d...
12693,ADEME,38529030900454,2019-10-09,1916C0010,ASSOCIAT TECHNIQUE ENERGIE ENVIRONNEMENT,3.150628e+13,action 2019 : activités associatives portant s...,186724.54,aide en numéraire,Unique,...,1.0,0.27,48.888082,2.241744,92,"Activités spécialisées, scientifiques et techn...",Activités des sièges sociaux ; conseil de gestion,Conseil de gestion,Conseil pour les affaires et autres conseils d...,Conseil pour les affaires et autres conseils d...
12694,ADEME,38529030900454,2019-07-08,1966C0092,SOLE JULIEN,4.297205e+13,"""VVE - Vers une Ville Ecomobile"" / Conception ...",103570.00,aide en numéraire,Echelonné,...,1.0,0.36,43.301622,5.382030,13,"Arts, spectacles et activités récréatives","Activités créatives, artistiques et de spectacle","Activités créatives, artistiques et de spectacle",Cré

In [3]:
# Value counts for every columns
for c in df_ademe.columns:
    print(df_ademe[c].value_counts())
    print("\n")

ADEME    12696
Name: Nom de l attribuant, dtype: int64


38529030900454    12696
Name: idAttribuant, dtype: int64


2020-11-26    150
2019-11-26    131
2019-11-28    130
2018-11-29    126
2020-11-24    104
             ... 
2019-12-12      1
2020-05-01      1
2019-02-01      1
2018-12-04      1
2019-11-30      1
Name: dateConvention, Length: 647, dtype: int64


1906C0018    10
1804C0025     9
2005C0015     8
1903C0003     8
2003C0001     7
             ..
2082C0008     1
2082C0009     1
2082C0010     1
2082C0011     1
20NOC0103     1
Name: referenceDecision, Length: 11795, dtype: int64


INSTITUT NATIONAL DE RECHERCHE POUR L'AGRICULTURE L'ALIMENTATION ET L'ENVIRONNEMENT    71
CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE                                           71
SUEZ RV REUNION                                                                        38
E. COMPAGNIE                                                                           35
SYNDICAT MIXTE POUR LA VALORISATION DES DECHE

On peut remarquer qu'il n'y a pas de numéro d'identification unique. La seule clé ici va être l'index. Attention alors si on récupère des données plus tard on ne pourat pas intersecter avec ces données. Il faudrat tout recommancer.

## Récupération des aides de Mission Transition.
Nous récupérons en particulier les offres de mission transition qui proviennent de l'ADEME.

In [4]:
def urlJsonToDataFrame(url):
    """
    This function return a pandas Dataframe using a given a json url.

            Parameter
            ---------
            url : sting
                An url which lead to a json

            Returns
            -------
            df : pandas DataFrame
                A pandas DataFrame of the json.
    """

    response = requests.get(url)
    response_json = response.json()
    df = pd.json_normalize(response_json)

    return df

In [5]:
df_mt_all = urlJsonToDataFrame(
    "https://mission-transition-ecologique.beta.gouv.fr/api/temp/aids"
)
df_mt_all

,name,perimeter,goal,beneficiary,aidDetails,eligibility,conditions,fundingSourceUrl,applicationEndDate,applicationUrl,...,subventionRateUpperBound,subventionRateLowerBound,loanAmount,applicationStartDate,projectExamples,directAccess,types,id,funder.name,funder.id
0,Financer les études et tests préalables aux in...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Études et tests pré...,<p>\n Critères d'éligibilité\n <br/>\n</p>\n<u...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<ul>\...,False,"[{'name': 'Aide financière', 'id': 1}]",189,ADEME,8
1,Soutenir financièrement le développement de l'...,NATIONAL,None,None,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,"<p>\n <strong>\n Sont éligibles, les dossiers...",None,https://appelsaprojets.ademe.fr/aap/ORPLAST202...,2022-09-15T00:00:00+00:00,,...,NaN,NaN,None,2020-09-22T00:00:00+00:00,<p>\n En cas de questions relatives à l'appel ...,False,"[{'name': 'Aide financière', 'id': 1}]",178,ADEME,8
2,Financer les études de préfiguration d'une dém...,NATIONAL,None,None,<p>\n Aide - « Étude de préfiguration d'une dé...,,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,69.0,NaN,None,2021-11-09T15:42:48+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<p>\n...,False,"[{'name': 'Aide financière', 'id': 1}]",203,ADEME,8
3,Réaliser des investissements de décarbonation ...,NATIONAL,None,None,<p>\n Plusieurs dispositifs sont ouverts aux e...,<p>\n <strong>\n Pour que la demande d'aide s...,None,https://www.asp-public.fr/aide-en-faveur-des-i...,2022-12-31T00:00:00+00:00,,...,NaN,NaN,None,2020-09-15T00:00:00+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",150,Ministère de la Transition Ecologique (MTE),52
4,Financer des audits énergétiques en entreprise...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Financement des aud...,<p>\n Critères d'éligibilité : u\n ne étude de...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",161,ADEME,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,Soutenir l'émergence de solutions innovantes p...,REGIONAL,None,None,<p>\n <strong>\n La Région soutient l'émergen...,,None,https://www.iledefrance.fr/soutien-aux-actions...,None,,...,NaN,NaN,None,2021-10-05T19:09:55+00:00,<p>\n <strong>\n Les projets peuvent exploite...,False,"[{'name': 'Aide financière', 'id': 1}]",280,Conseil régional d'Ile de France,44
383,Sortir de vacance les logements locatifs privés,REGIONAL,None,None,<p>\n Le présent dispositif vise à remettre su...,<p>\n <strong>\n Bénéficiaires éligibles :\n ...,None,,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",96,Conseil départemental de la Vendée,41
384,Soutenir l'économie de proximité en finançant ...,REGIONAL,None,None,<p>\n <strong>\n Aide - « Fonds régional des ...,,None,https://www.bourgognefranchecomte.fr/node/2835,2021-12-31T00:00:00+00:00,,...,NaN,NaN,None,2021-11-09T15:42:40+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",59,Conseil régional de Bourgogne-Franche-Comté,5
385,Soutenir le développement d'investissements ag...,REGIONAL,None,None,<p>\n La Région Occitanie est aux côtés des ac...,<p>\n <strong>\n Bénéficiaires\n </strong>\n ...,None,https://www.laregion.fr/PASS-Agritourisme,None,,...,NaN,NaN,None,2021-11-09T15:42:40+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",64,Conseil régional d'Occitanie,9


In [6]:
# Aids from ADEME into mission transition dataset
ademe_aids = df_mt_all["funder.name"] == "ADEME"
df_mt = df_mt_all[ademe_aids]
df_mt

,name,perimeter,goal,beneficiary,aidDetails,eligibility,conditions,fundingSourceUrl,applicationEndDate,applicationUrl,...,subventionRateUpperBound,subventionRateLowerBound,loanAmount,applicationStartDate,projectExamples,directAccess,types,id,funder.name,funder.id
0,Financer les études et tests préalables aux in...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Études et tests pré...,<p>\n Critères d'éligibilité\n <br/>\n</p>\n<u...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<ul>\...,False,"[{'name': 'Aide financière', 'id': 1}]",189,ADEME,8
1,Soutenir financièrement le développement de l'...,NATIONAL,None,None,<p>\n Le dispositif de l'ADEME – ORPLAST (Obje...,"<p>\n <strong>\n Sont éligibles, les dossiers...",None,https://appelsaprojets.ademe.fr/aap/ORPLAST202...,2022-09-15T00:00:00+00:00,,...,NaN,NaN,None,2020-09-22T00:00:00+00:00,<p>\n En cas de questions relatives à l'appel ...,False,"[{'name': 'Aide financière', 'id': 1}]",178,ADEME,8
2,Financer les études de préfiguration d'une dém...,NATIONAL,None,None,<p>\n Aide - « Étude de préfiguration d'une dé...,,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,69.0,NaN,None,2021-11-09T15:42:48+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<p>\n...,False,"[{'name': 'Aide financière', 'id': 1}]",203,ADEME,8
4,Financer des audits énergétiques en entreprise...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Financement des aud...,<p>\n Critères d'éligibilité : u\n ne étude de...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,70.0,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",161,ADEME,8
5,Financer les équipements de lutte contre le ga...,NATIONAL,None,None,<p>\n <strong>\n Aide - « Équipements de lutt...,<p>\n Critères d'éligibilité :\n <br/>\n <ul>\...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,55.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Vous pouvez prétendre à une aide pour le...,False,"[{'name': 'Aide financière', 'id': 1}]",190,ADEME,8
9,Gérer des biodéchets des acteurs économiques -...,NATIONAL,None,None,<p>\n <em>\n Mieux gérer vos biodéchets : l'A...,<p>\n Vous êtes une entreprise de restauration...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide en ingénierie', 'id': 2}]",39,ADEME,8
10,Investir pour la gestion des biodéchets des ac...,NATIONAL,None,None,"<p>\n <em>\n Pour mieux gérer les biodéchets,...",<p>\n Vous êtes une entreprise de restauration...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",43,ADEME,8
11,Aider au déploiement du tri sélectif hors foyer,NATIONAL,None,None,<p>\n <strong>\n Aide - « Déploiement du tri ...,<p>\n <strong>\n Critères d'éligibilité :\n <...,None,https://agirpourlatransition.ademe.fr/entrepri...,2022-06-30T00:00:00+00:00,,...,50.0,NaN,None,2021-11-09T15:42:41+00:00,<p>\n Actions éligibles : les investissements ...,False,"[{'name': 'Aide financière', 'id': 1}]",177,ADEME,8
12,Subventionner des études préalables à la réali...,NATIONAL,None,None,<p>\n <em>\n Réalisation d'une étude territor...,<p>\n <strong>\n L'aide s'adresse aux acteurs...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",42,ADEME,8
13,Investir dans les centres de tri et de prépara...,NATIONAL,None,None,<p>\n <em>\n Vous souhaitez réaliser les inve...,<p>\n L'aide s'adresse aux acteurs publics ou ...,None,https://agirpourlatransition.ademe.fr/entrepri...,None,,...,NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",41,ADEME,8


Ici nous choisissons la clé : id. C'est domage que la colonne source_id qui était dans les premiers fichier n'apparais plus ici car elle correspond à l'id de aide territoire, dataset plus large si besoin.

## Lien entre les offres de l'ADEME et celles de Mission transition

La colonne objet des offres de l'ADEME et la colonne name des offres de Mission Transition sembles toutes les deux indiquer l'intitulé du projet. Cependant il n'existent pas de lien direct au sens ou l'intitulé n'est jamais exactement le même. Cependant, avec un examen plus minutieux on retrouve pratiquement les mêmes appellations. Voir les exemples ci dessous.

In [7]:
# inclusion
ademe_1 = df_ademe.objet.iloc[0]
mt_1 = df_mt.name.iloc[21]  # id 44

print(ademe_1)
print(mt_1)

TREMPLIN pour la transition écologique des PME
Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME »


In [8]:
# Closeness
ademe_2 = df_ademe.objet.iloc[1582]
mt_2 = df_mt.name.iloc[29]  # id 44

print(ademe_2)
print(mt_2)

Réalisation d'une installation de production d'eau chaude solaire à la Maison de retraite des 5 étan
Financer les installations de production d'eau chaude solaire


Dans la première situation il suffit de chercher simplement la presence d'une chaîne de caractère dans l'autre. Dans le second cas nous nous proposons d'utiliser la distance de Levenshtein.

In [9]:
# Inclusion
ademe_1.lower() in mt_1.lower()

True

In [10]:
# Closeness
print(fuzz.ratio(ademe_2, mt_2))
print(fuzz.partial_ratio(ademe_2, mt_2))
print(fuzz.token_sort_ratio(ademe_2, mt_2))

66
87
64


In [11]:
# Just a first test !

In [12]:
aids_ademe = df_ademe.objet.values
aids_mt = df_mt.name.values

In [13]:
i = 0
Levenshtein_threshold = 60
for ademe_name in aids_ademe:
    for mt_name in aids_mt:

        ratio = fuzz.ratio(ademe_name, mt_name)

        if ratio > Levenshtein_threshold:
            i += 1
            print(ratio)
            print(ademe_name)
            print(mt_name)
            print("\n")

64
Mise en oeuvre du contrat patrimonial de développement des énergies renouvelables thermiques
Développer les énergies renouvelables thermiques


62
contrat de développement Patrimonial des énergies renouvelables thermiques de la SCIC BEL dans le Lo
Développer les énergies renouvelables thermiques


63
contrat de développement patrimonial des énergies renouvelables thermiques sur quatre installations
Développer les énergies renouvelables thermiques


61
réalisation d'une installation géothermique sur nappe assistée par une pompe à chaleur de 270 kW pou
Financer les installations géothermiques de production de chaleur et de froid


61
AAP STCo - Installation de production d'eau chaude solaire pour 10 villas (24 m²)
Aider à l'installation de production de chaleur biomasse/bois


68
AAP STCo - Installation de production d'eau chaude solaire pour 10 villas (24 m²)
Financer les installations de production d'eau chaude solaire


63
Aide à l'investissement pour la valorisation de déchets de 

67
Contrat de développement territorial des énergies renouvelables thermiques en Lozère (48)
Développer les énergies renouvelables thermiques


61
Mission d'assistance à maîtrise d'ouvrage pour la création d'un réseau de chaleur au bois, destiné à
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


62
Réhabilitation de deux installations collectives de production d'eau chaude solaire résidence "Le Cl
Financer les installations géothermiques de production de chaleur et de froid


68
Réhabilitation de deux installations collectives de production d'eau chaude solaire résidence "Le Cl
Financer les installations de production d'eau chaude solaire


61
Mise en place d'une installation géothermique sur champs de sondes verticales permettant le chauffag
Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments


61
Mise en place d'une installation géothermique sur champs de sondes verticales permettant le chauffag
Financer les installations 

68
Etude de faisabilité d'une chaudière bois et installation solaire thermique à Gesté (49)
Financer une étude de faisabilité pour une installation solaire thermique


62
Installation d'une PAC géothermique sur champ de sondes pour une cave viticole à Beaulieu sur Layon
Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments


62
Evaluation de la performance environnementale d'une offre servicielle de réparation de vêtements et
Améliorer la performance environnementale des produits et des services - Études d’écoconception


64
Evaluation de la performance environnementale d'une offre servicielle de réparation de vêtements et
Améliorer la performance environnementale des produits et services - Investissements d’écoconception


66
Contrat de développement des énergies renouvelables thermiques : Convention d'Animation.
Développer les énergies renouvelables thermiques


63
AURAGEOTH 2020_Réalisation d'une installation de géothermie sur champ de sondes pour un

61
étude de faisabilité d'un réseau de chaleur bois énergie communal
Financer les études de faisabilité de chauffage des serres par énergie renouvelable


61
étude de faisabilité d'un réseau de chaleur bois énergie communal
Financer une étude de faisabilité de récupération de chaleur fatale


66
la réalisation d'une étude de faisabilité d'un réseau de chaleur alimenté par des énergies renouvela
Financer les études de faisabilité de chauffage des serres par énergie renouvelable


64
AAP Economie circulaire : Mise en place d'équipements de lutte contre le gaspillage alimentaire
Financer les équipements de lutte contre le gaspillage


61
mission d'assistance à maitrise d'ouvrage pour le projet du centre multimodal de Bastia
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


67
Etude de faisabilité d'une installation de réemploi d'aides techniques
Financer une étude de faisabilité pour une installation solaire thermique


61
Etude de faisabilité et Assistance à Mai

66
la réalisation d'une étude de récupération et de valorisation de chaleur fatale
Financer une étude de faisabilité de récupération de chaleur fatale


62
réalisation d'une étude de faisabilité pour la création d'un réseau de chaleur et d'un réseau de fro
Financer une étude de faisabilité de récupération de chaleur fatale


74
AAP ST - Etude de faisabilité pour l'installation de panneaux solaires thermiques
Financer une étude de faisabilité pour une installation solaire thermique


65
AAP ST - Etude de faisbilité pour une installation de production d'eau chaude solaire
Financer une étude de faisabilité pour une installation solaire thermique


71
AAP ST - Etude de faisbilité pour une installation de production d'eau chaude solaire
Financer les installations de production d'eau chaude solaire


67
Etude de faisabilité solaire thermique
Financer une étude de faisabilité pour une installation solaire thermique


64
Étude de projet et Assistance à Maitrise d'Ouvrage pour un réseau de chal

62
Financer l'étude de faisabilité pour le moulin des Prés à Auxerre sur l'Yonne (89)
Financer une étude de faisabilité pour une installation solaire thermique


61
Financer l'étude de faisabilité pour le moulin des Prés à Auxerre sur l'Yonne (89)
Financer les études de faisabilité de chauffage des serres par énergie renouvelable


63
FONDS CHALEUR : Etude de faisabilité pour la récupération de chaleur sur les fours de séchage et de
Financer une étude de faisabilité de récupération de chaleur fatale


67
FONDS CHALEUR : Assistance à maîtrise d'ouvrage pour la création d'une chaufferie bois avec réseau d
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


69
FONDS CHALEUR : Réalisation d'une assistance à maîtrise d'ouvrage pour la création d'une chaufferie
Financer une assistance à maîtrise d’ouvrage pour une chaufferie biomasse


63
Réaliser une étude de faisabilité géothermie.
Financer une étude de faisabilité pour une installation solaire thermique


62
Réalis

61
Réalisation d'une étude de faisabilité géothermie
Financer une étude de faisabilité pour une installation solaire thermique


82
Réalisation d'une étude de faisabilité de récupération de chaleur fatale pour la SVEP
Financer une étude de faisabilité de récupération de chaleur fatale


61
Etude de faisabilité Hydrogéologique pour mise en place d'une installation géothermique - Phase 1
Financer une étude de faisabilité pour une installation solaire thermique


66
Etude de faisabilité d'un réseau de chaleur biomasse
Financer une étude de faisabilité de récupération de chaleur fatale


74
Réalisation d'une étude de faisabilité récupération de chaleur du groupe froid
Financer une étude de faisabilité de récupération de chaleur fatale


68
Réalisation d'une étude de faisabilité récupération de chaleur sur tours aéroréfrigérantes et compre
Financer une étude de faisabilité de récupération de chaleur fatale


65
Etude de faisabilité réseau de chaleur biomasse
Financer une étude de faisabilit

66
Etude de faisabilité de récupération de chaleur fatale sur le site de Caudebec Les Elbeuf (76)
Financer une étude de faisabilité de récupération de chaleur fatale


63
Cadre d'intervention EIT - Démarche d'écologie industrielle et territoriale sur le territoire de la
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


72
Développement de la démarche d'écologie industrielle et territorial
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


66
Accompagnement à la mise en place d'une démarche d'écologie Industrielle et Territoire sur la zone d
Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale


62
Accompagnement à la mise en place d'une démarche d'écologie Industrielle et Territoire sur la zone d
Accompagner les entreprises dans la mise en œuvre d'une démarche d'écoconception de leurs produits et services numériques


62
Création d'une conserverie locale solidai

In [14]:
i

420

## Création d'un dataset entre ADEME et Mission Transition
Dans cette partie nous allons créer un dataset en faisant matcher des aides de l'ADEME avec celle de Mission Transition. Pour ce faire nous allons dans un premier temps créer un dataset de scores, qui a chaque paire d'intitulé d'offre d'aide associe les scores de Levenshtein proposés par la bibliothèque fuzzywuzzy. Dans un second temps nous prosons une façon de matcher à partir des score de Leveinshtein. Enfin, nous créérons le dataset cobinant les données de l'ADEME et de Mission Transition.

### Calcul des scores de Levenshtein

In [15]:
# Data from ademe
df_ademe_ = pd.DataFrame(
    data={"ademe_objet": df_ademe["objet"], "ademe_index": df_ademe.index}
)

# Data from mission transition
df_mt_ = pd.DataFrame(data={"mt_name": df_mt["name"], "mt_id": df_mt["id"]})

# Product of ademe and mission transition
df_product = df_ademe_.merge(df_mt_, how="cross")

In [16]:
df_product

,ademe_objet,ademe_index,mt_name,mt_id
0,TREMPLIN pour la transition écologique des PME,0,Financer les études et tests préalables aux in...,189
1,TREMPLIN pour la transition écologique des PME,0,Soutenir financièrement le développement de l'...,178
2,TREMPLIN pour la transition écologique des PME,0,Financer les études de préfiguration d'une dém...,203
3,TREMPLIN pour la transition écologique des PME,0,Financer des audits énergétiques en entreprise...,161
4,TREMPLIN pour la transition écologique des PME,0,Financer les équipements de lutte contre le ga...,190
...,...,...,...,...
634795,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Financer les études liées à la réduction des d...,184
634796,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Accompagner tous les acteurs ligériens dans la...,9
634797,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Financer des études pour les collectivités, as...",174
634798,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Développer un projet de recherche, de développ...",140


In [17]:
def LevenshteinScores(n1, n2):
    """
    This function return a dictionary of differents levenshtein scores given by
    the fuzzywuzzy library.

            Parameter
            ---------
            n1 : sting
                String to compare with n2

            n2 : sting
                String to compare with n1

            Returns
            -------
            scores_dict : dictionary
                A dictionary of differents scores

    """
    n_1 = n1.lower()
    n_2 = n2.lower()

    l = fuzz.ratio(n_1, n_2)
    l_partial = fuzz.partial_ratio(n_1, n_2)
    l_token_sort = fuzz.token_sort_ratio(n_1, n_2)
    l_token_set = fuzz.token_set_ratio(n_1, n_2)

    score_dict = {
        "L": l,
        "L_partial": l_partial,
        "L_token_sort": l_token_sort,
        "L_token_set": l_token_set,
    }

    return score_dict

In [18]:
def saveDataFrame(df, folder, file_name):
    """
    This function save the DataFrame df as csv into the folder/file_name.

            Parameter
            ---------
            df : A Pandas DataFrame
                A DataFrame to save.

            folder : sting
                The folder where to save.

            file_name : sting
                The name of the file to save.
    """

    path_to_data = os.path.join(folder, file_name)
    df.to_csv(path_to_data, index=False)


def loadDataFrame(folder, file_name):
    """
    This function return a Pandas DataFrame load from the file folder/file_name.

            Parameter
            ---------
            folder : sting
                Folder where to get the DataFrame.

            file_name : sting
                file_name of a csv to load.

            Returns
            -------
            df : A Pandas DataFrame
                The Pandas DataFrame loaded from the csv file

    """
    path_to_data = os.path.join(folder, file_name)
    return pd.read_csv(path_to_data)

In [19]:
RECOMPUTE = False
SAVE = False
if RECOMPUTE:
    # Can takes few minutes to compute the Levenstein scores on the entire dataset.
    df_levenshtein = df_product.apply(
        lambda x: LevenshteinScores(x["ademe_objet"], x["mt_name"]),
        axis=1,
        result_type="expand",
    )

    df_scores = df_product.join(df_levenshtein)

else:
    df_scores = loadDataFrame(DATA_FOLDER, "Matching_scores.csv")

if SAVE:

    saveDataFrame(df_scores, DATA_FOLDER, "Matching_scores.csv")

In [20]:
df_scores

,ademe_objet,ademe_index,mt_name,mt_id,L,L_partial,L_token_sort,L_token_set
0,TREMPLIN pour la transition écologique des PME,0,Financer les études et tests préalables aux in...,189,32,48,33,35
1,TREMPLIN pour la transition écologique des PME,0,Soutenir financièrement le développement de l'...,178,35,41,37,37
2,TREMPLIN pour la transition écologique des PME,0,Financer les études de préfiguration d'une dém...,203,39,46,45,46
3,TREMPLIN pour la transition écologique des PME,0,Financer des audits énergétiques en entreprise...,161,38,43,39,39
4,TREMPLIN pour la transition écologique des PME,0,Financer les équipements de lutte contre le ga...,190,36,37,39,39
...,...,...,...,...,...,...,...,...
634795,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Financer les études liées à la réduction des d...,184,37,42,41,40
634796,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,Accompagner tous les acteurs ligériens dans la...,9,38,42,46,46
634797,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Financer des études pour les collectivités, as...",174,41,51,43,45
634798,"""VVE - Vers une Ville Ecomobile"" / Conception ...",12695,"Développer un projet de recherche, de développ...",140,39,54,41,49


### Premier match automatique
Pour l'instant on se propose de réaliser un premier match sur les aides à l'aide d'un seuil sur les scores. Si au moins un score dépasse le seuil on match le couple d'aides.

In [28]:
def firstMatch(L, L_partial, L_token, L_token_set, Threshold):
    """
    This function return a dictionary of differents Levenshtein scores given by
    the fuzzywuzzy library.

            Parameter
            ---------
            L, L_partial, L_token, L_token_set, : int
                Differents score from the fuzzywuzzy library

            Threshold : int
                Threshold to copmare to the differents scores.

            Returns
            -------
            match_dict : dictionary of booleans
                A dictionary telling us if there is a match or not

    """
    if max(L, L_partial, L_token, L_token_set) > Threshold:
        return {"match": True}
    else:
        return {"match": False}

In [22]:
# We get around 1000 match for a threshold equal to 70
threshold = 75

df_match = df_scores.apply(
    lambda x: firstMatch(
        x["L"], x["L_partial"], x["L_token_sort"], x["L_token_set"], threshold
    ),
    axis=1,
    result_type="expand",
)

matched = df_match == True
df_first_match = df_scores[matched.values]

A la fin on veut qu'il y ait au plus une aide de Mission Transition associé à une chaque aide de l'ADEME. Pour l'instant ça n'est pas le cas. Nous pourrions automatiser ce processus en prennant par exemple le meileur score de tout les types de score mais il reste cependant pas mal d'erreur. Voir le code ci-dessous.

In [30]:
def displayMultipleMatchs(df):
    """
    This function show informations for every match of the matching DataFrame
    df. For every match we show the ADEME and Mission Transition names, and the
    Levenshtein scores for the matching.

            Parameter
            ---------
            df : A pandas DataFrame
                A pandas DataFrame with Levenshtein scores.
    """
    for index, row in df.iterrows():
        same_ademe = []

        for i, r in df.iterrows():
            if row["ademe_index"] == r["ademe_index"]:
                same_ademe.append(r)

        print("ADEME : {}".format(row.ademe_objet))
        [
            print(
                "MT match : {} \nscores : {},{},{},{}.".format(
                    r.mt_name, r.L, r.L_partial, r.L_token_sort, r.L_token_set
                )
            )
            for r in same_ademe
        ]
        print("\n")

In [31]:
displayMultipleMatchs(df_first_match)

ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets d

ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets d

ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets de transition écologique avec les aides financières du guichet « Tremplin pour la transition écologique des PME » 
scores : 51,100,52,100.


ADEME : TREMPLIN pour la transition écologique des PME
MT match : Financer vos projets d

ADEME : Construction d'une unité de méthanisation à Peyrat de Bellac (87)
MT match : Financer des « Études préalables à la construction d’une installation de méthanisation » 
scores : 48,57,62,76.


ADEME : Construction d'une unité de méthanisation à Azat-le-Ris (87)
MT match : Financer des « Études préalables à la construction d’une installation de méthanisation » 
scores : 50,62,62,77.


ADEME : Contrat de développement des EnR thermiques - Développement des énergies renouvelables sur le départ
MT match : Développer les énergies renouvelables thermiques 
scores : 54,75,64,82.


ADEME : Contrat de développement territorial des énergies renouvelables thermiques sur le département de la
MT match : Développer les énergies renouvelables thermiques 
scores : 64,83,62,82.


ADEME : Projet DTR
MT match : Accompagner les projets des entreprises engagées dans une transition pour une mode durable 
scores : 20,80,20,21.
MT match : Financer vos projets de transition écologique avec les aides fina

ADEME : PERFECTO 2019 - Projet CIBLE PSF - Améliorer la performance environnementale des produits d'épilatio
MT match : Améliorer la performance environnementale des produits et des services - Études d’écoconception 
scores : 63,76,68,78.
MT match : Améliorer la performance environnementale des produits alimentaires (GREEN GO) 
scores : 66,83,65,83.


ADEME : PERFECTO 2019 - Projet CIBLE PSF - Améliorer la performance environnementale des produits d'épilatio
MT match : Améliorer la performance environnementale des produits et des services - Études d’écoconception 
scores : 63,76,68,78.
MT match : Améliorer la performance environnementale des produits alimentaires (GREEN GO) 
scores : 66,83,65,83.


ADEME : Poste d'animateur Ecologie  Industrielle et Territoriale
MT match : Financer les études de préfiguration d'une démarche d'écologie industrielle et territoriale 
scores : 64,79,56,71.


ADEME : Construction d'une unité de méthanisation aux Fins (25)
MT match : Financer des « Études pr

...

ADEME : Etude de faisabilité bois énergie
MT match : Financer une étude de faisabilité pour une installation solaire thermique 
scores : 55,76,37,59.
MT match : Financer les études de faisabilité de chauffage des serres par énergie renouvelable 
scores : 52,70,45,78.


ADEME : Etude de faisabilité bois énergie
MT match : Financer une étude de faisabilité pour une installation solaire thermique 
scores : 55,76,37,59.
MT match : Financer les études de faisabilité de chauffage des serres par énergie renouvelable 
scores : 52,70,45,78.


ADEME : la réalisation d'une étude de faisabilité géothermique pour le logement communal dans le cadre du Contrat de Développement des Energies Renouvelables "Sud Berry"
MT match : Développer les énergies renouvelables thermiques 
scores : 35,77,36,44.


ADEME : la réalisation d'une étude de faisabilité de récupération de chaleur fatale du four TTH HTI pour préchauffer les pièces en amont du four ou d'un four de revenu.
MT match : Financer une étude de fai

ADEME : Etude de faisabilité de mise en oeuvre d'un contrat de développement des Energies renouvelables ther
MT match : Développer les énergies renouvelables thermiques 
scores : 53,84,53,57.


ADEME : Réalisation d'une étude de faisabilité solaire thermique pour la Résidence "Le Moulin de la Cour" à Saint Pierre du Lorouër  (72).
MT match : Financer une étude de faisabilité pour une installation solaire thermique 
scores : 50,63,60,80.


ADEME : Etude stratégique du développement des énergies renouvelables sur le territoire de Laval Agglo (53)
MT match : Développer les énergies renouvelables thermiques 
scores : 56,79,59,63.


ADEME : Réalisation d'une étude de faisabilité pour une installation de PAC géothermale pour la Maison saint Vincent de Paul à Saint-Etienne (42).
MT match : Financer une étude de faisabilité pour une installation solaire thermique 
scores : 58,79,55,75.


ADEME : Réalisation d'une étude de faisabilité pour la création d'un réseau de chaleur alimenté par une cha

ADEME : FONDS CHALEUR : Réalisation d'une étude de faisabilité  pour l'installation d'une chaufferie biomasse afin d'équiper le nouveau bâtiment en énergie renouvelable.
MT match : Aider à l'installation de production de chaleur biomasse/bois 
scores : 40,64,48,76.
MT match : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur) 
scores : 50,60,54,83.


ADEME : FONDS CHALEUR : Etude de faisabilité de récupération de chaleur.
MT match : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur) 
scores : 63,61,63,76.
MT match : Financer une étude de faisabilité de récupération de chaleur fatale 
scores : 81,83,68,84.


ADEME : FONDS CHALEUR : Etude de faisabilité de récupération de chaleur.
MT match : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur) 
scores : 63,61,63,76.
MT match : Financer une étude de faisabilité de récupération de chaleur fatale 
scores : 81,83,68,84.


ADEME : FONDS CHALEUR : Etude de faisabilité pour la récupérati

ADEME : 3 études de faisabilité de réseaux de chaleur biomasse
MT match : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur) 
scores : 60,69,75,89.
MT match : Financer une étude de faisabilité de récupération de chaleur fatale 
scores : 69,76,63,67.


ADEME : 3 études de faisabilité de réseaux de chaleur biomasse
MT match : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur) 
scores : 60,69,75,89.
MT match : Financer une étude de faisabilité de récupération de chaleur fatale 
scores : 69,76,63,67.


ADEME : Réalisation d'une étude de faisabilité solaire thermique pour la production d'eau chaude pour un éle
MT match : Financer une étude de faisabilité pour une installation solaire thermique 
scores : 57,63,68,80.


ADEME : étude de faisabilité d'une récupération de chaleur fatale sur l'extraction d'air en sortie des chemi
MT match : Financer une étude de faisabilité de récupération de chaleur fatale 
scores : 65,81,69,91.


ADEME : Etude de faisabi

ADEME : Création d'une conserverie locale solidaire permettant de lutter contre le gaspillage alimentaire
MT match : Financer les équipements de lutte contre le gaspillage 
scores : 62,76,56,63.


ADEME : Création d'une déchèterie professionnelle
MT match : Subventionner la construction et moderniser des déchèteries professionnelles 
scores : 63,78,62,62.
MT match : Financer des études préalables aux investissements dans les déchetteries professionnelles 
scores : 57,76,50,50.


ADEME : Création d'une déchèterie professionnelle
MT match : Subventionner la construction et moderniser des déchèteries professionnelles 
scores : 63,78,62,62.
MT match : Financer des études préalables aux investissements dans les déchetteries professionnelles 
scores : 57,76,50,50.


ADEME : Création d'une déchèterie professionnelle
MT match : Subventionner la construction et moderniser des déchèteries professionnelles 
scores : 63,78,62,62.
MT match : Financer des études préalables aux investissements dans l

In [32]:
# Let's save the obtained match. We don't need the scores anymore.
df_match = df_first_match[["ademe_objet", "ademe_index", "mt_name", "mt_id"]]
saveDataFrame(df_match, DATA_FOLDER, "First_match.csv")

### Rafinement à la main des matchs obtenus automatiquement.

On se propose donc de créer un dataset, en corrigeant à la main les quelques erreur et match multiples du dataset df_first_match. On n'a donc ici plus besoin des colones de scores. On créer quelques outils pour faciliter le raffinement à la main.

In [34]:
def displayDfMatch(df):
    """
    This function show informations for every match of the matching DataFrame
    df. For every match we show the index of df, the ADEME and Mission
    Transition names, and the respectives index.

            Parameter
            ---------
            df : A pandas DataFrame
                A pandas DataFrame to display with names and index.
    """

    for i, r in df.iterrows():
        print("match_index : {}".format(i))
        print("ademe_index : {}".format(r.ademe_index))
        print("ademe_objet : {}".format(r.ademe_objet))
        print("mt_name : {}".format(r.mt_name))
        print("mt_id : {}".format(r.mt_id))
        print("\n")

In [37]:
def changeMtInsideDf(ademe_index, new_mt_id, df):
    """
    This function change the Mission Transition columns "mt_id" and "mt_name"
    inside df for every row whith a given ademe_index.

            Parameter
            ---------
            ademe_index : An integer
                The ademe index where to remplace the Mission Transition information.

            new_mt_id : An integer
                The index of the Mission Transition to remplace in df.

            df : A Pandas DataFrame
                A pandas DataFrame with ADEME and MT names and id.

    """
    for i, r in df.iterrows():
        if r.ademe_index == ademe_index:
            r.mt_id = new_mt_id
            r.mt_name = mtNameFromId(new_mt_id)
            df.loc[i] = r


def mtNameFromId(mt_id):
    """
    This function return the name of a Mission Transition aid offer given by
    its id.

            Parameter
            ---------
            new_mt_id : An integer
                The index of the Mission Transition to return

            Return
            ------
            mt_name : a string
                The Mission Transition name coresponding to the id mt_id

    """
    for i, r in df_mt.iterrows():
        if r.id == mt_id:
            mt_name = r["name"]
            return mt_name


def deleteRowByIndex(df, match_index):
    """
    This function delete the row of index match_index inside a matching
    DataFrame df.

            Parameter
            ---------
            match_index : an integer
                The index of the row to delete.

            df : A Pandas DataFrame
                A pandas DataFrame where to delete one row.

    """
    df = df.drop(match_index)
    return df

In [33]:
df_test = loadDataFrame(DATA_FOLDER, "Handmade_match.csv")

In [38]:
# display
index_floor = 455
displayDfMatch(df_test[index_floor:])

match_index : 455
ademe_index : 11111
ademe_objet : Etude de faisabilité Chaleur Fatale à Chatillon sur Chalaronne (01)  /
mt_name : Financer une étude de faisabilité de récupération de chaleur fatale
mt_id : 268


match_index : 456
ademe_index : 11126
ademe_objet : Réalisation d'une étude de faisabilité pour une installation de géothermie sur sondes pour la rénova
mt_name : Tester l’intérêt de la géothermie sur champs de sondes pour le chauffage des bâtiments
mt_id : 269


match_index : 457
ademe_index : 11178
ademe_objet : Etude Chaufferie bois
mt_name : Financer des études de faisabilité chaufferie biomasse (Fonds chaleur)
mt_id : 266


match_index : 458
ademe_index : 11186
ademe_objet : Etudes préalables investissements déchèterie professionnelle à Baneins (01)
mt_name : Financer des études préalables aux investissements dans les déchetteries professionnelles
mt_id : 180


match_index : 459
ademe_index : 11234
ademe_objet : Installation solaire thermique ECS "Pavade"
mt_name : Fina

In [39]:
# Change the mt associated to a ademe index.
ademe_index = 11349
new_mt_id = 266

changeMtInsideDf(ademe_index, new_mt_id, df_test)

In [40]:
# Quick print of the mt aids offers
L = [(r.id, r["name"]) for i, r in df_mt.iterrows()]
[print("id : {}\nname : {}\n".format(l[0], l[1])) for l in sorted(L)]

id : 9
name : Accompagner tous les acteurs ligériens dans la mise en œuvre du tri à la source des biodéchets en Pays de la Loire -BioD PDL

id : 34
name : Financer les études d’écoconception des produits et des services (secteurs du numérique, de la mode, de l’alimentation, et tout autre secteur)

id : 37
name : Améliorer la performance environnementale des produits et services - Investissements d’écoconception

id : 38
name : Améliorer la performance environnementale des produits et des services - Études d’écoconception

id : 39
name : Gérer des biodéchets des acteurs économiques - Études préalables aux investissements

id : 40
name : Subventionner la construction et moderniser des déchèteries professionnelles

id : 41
name : Investir dans les centres de tri et de préparation des déchets pour une valorisation matière et énergie

id : 42
name : Subventionner des études préalables à la réalisation de centres de tri/préparation des déchets pour une valorisation matière et énergie

id : 4

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [47]:
# match_index = 466
# df_test = deleteRowByIndex(df_test, match_index)

# saveDataFrame(df_test,DATA_FOLDER,"Handmade_match.csv")

### Création d'un dataset des données de l'ADEME et de Mission Transition.
Ici, il ne reste plus qu'a faire un gros dataset pour lequel chaque ligne correspond à une offre d'aide de l'ADEME qui a abouti à un projet, et les colones correspondent à la fois au informations sur l'entreprise qui proviennent des données de l'ADEME, et à la fois des données de mission transition de l'aide associée grâce au matching.

In [41]:
df_match = loadDataFrame(DATA_FOLDER, "Handmade_match.csv")

In [42]:
def createAdemeMtDataset(df_match, df_ademe, df_mt):
    """
    Create a concatenated DataFrame from the df_ademe and df_mt columns by
    using the matchs from df_match.

            Parameter
            ---------
            df_match : A Pandas DataFrame
                A pandas DataFrame of match with for every row an index from
                ADEME and and index from MT.

            df_ademe : A Pandas DataFrame
                A pandas DataFrame of features from ADEME.

            df_mt : A Pandas DataFrame
                A pandas DataFrame of features from MT.


    """
    # Get data from ademe
    df_match_ademe = df_ademe.loc[df_match.ademe_index]
    df_match_ademe.reset_index(inplace=True)
    addWordColNames(df_match_ademe, "ademe")

    # Get data from mt
    df_mt_ = df_mt.set_index("id")
    df_match_mt = df_mt_.loc[df_match.mt_id]
    df_match_mt.reset_index(inplace=True)
    addWordColNames(df_match_mt, "mt")

    # Union
    df_all = df_match_ademe.join(df_match_mt)

    return df_all


def addWordColNames(df, w):
    """This fuction add a word w at every columns names of the Pandas DataFrame
    df.

            Parameter
            ---------
            df : A Pandas DataFrame
            A Pandas DataFrame where we change every columns names.

            w : string
            A word to add to every column names of the DataFrame df."""
    columns = df.columns
    new_columns = [c + "_" + w for c in columns]
    df.columns = new_columns

In [43]:
df_new = createAdemeMtDataset(df_match, df_ademe, df_mt)
df_new

,index_ademe,Nom de l attribuant_ademe,idAttribuant_ademe,dateConvention_ademe,referenceDecision_ademe,nomBeneficiaire_ademe,idBeneficiaire_ademe,objet_ademe,montant_ademe,nature_ademe,...,contactGuidelines_mt,subventionRateUpperBound_mt,subventionRateLowerBound_mt,loanAmount_mt,applicationStartDate_mt,projectExamples_mt,directAccess_mt,types_mt,funder.name_mt,funder.id_mt
0,0,ADEME,38529030900454,2021-04-12,21BFD0081,CLEMENT SAS,7.269205e+13,TREMPLIN pour la transition écologique des PME,6500.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:49+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",ADEME,8
1,1,ADEME,38529030900454,2021-03-25,21BFD0119,INDUSTRIE DOLOISE DE MICRO-MECANIQUE,3.950618e+13,TREMPLIN pour la transition écologique des PME,12150.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:49+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",ADEME,8
2,2,ADEME,38529030900454,2021-03-23,21BFD0123,TRANSDEV BFC SUD,3.391339e+13,TREMPLIN pour la transition écologique des PME,16000.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:49+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",ADEME,8
3,3,ADEME,38529030900454,2021-03-23,21BFD0137,LOCA TRAVAUX EURL,4.999836e+13,TREMPLIN pour la transition écologique des PME,16400.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:49+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",ADEME,8
4,4,ADEME,38529030900454,2021-03-26,21BFD0142,RODESCHINI SAS,3.008465e+13,TREMPLIN pour la transition écologique des PME,21500.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:49+00:00,,False,"[{'name': 'Aide financière', 'id': 1}, {'name'...",ADEME,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,12304,ADEME,38529030900454,2020-12-03,20REC0006,SYNERGIE PEI,8.891104e+13,Développement de la démarche d'écologie indust...,181800.0,aide en numéraire,...,"<p>\n Contactez l'ADEME via leur\n <a href=""ht...",69.0,NaN,None,2021-11-09T15:42:48+00:00,<p>\n Actions éligibles :\n <br/>\n</p>\n<p>\n...,False,"[{'name': 'Aide financière', 'id': 1}]",ADEME,8
470,12587,ADEME,38529030900454,2018-08-27,18GEC0080,PARTAGE TON FRIGO,7.925276e+13,Création d'une conserverie locale solidaire pe...,44406.0,aide en numéraire,...,"<p>\n Pour en savoir plus sur ce dispositif, c...",55.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Vous pouvez prétendre à une aide pour le...,False,"[{'name': 'Aide financière', 'id': 1}]",ADEME,8
471,12588,ADEME,38529030900454,2018-08-27,18GEC0080,PARTAGE TON FRIGO,7.925276e+13,Création d'une conserverie locale solidaire pe...,44406.0,aide en numéraire,...,"<p>\n Pour en savoir plus sur ce dispositif, c...",55.0,NaN,None,2021-11-09T15:42:42+00:00,<p>\n Vous pouvez prétendre à une aide pour le...,False,"[{'name': 'Aide financière', 'id': 1}]",ADEME,8
472,12605,ADEME,38529030900454,2018-05-30,18PLC0023,SCI SAINT MICHEL,7.892967e+13,Création d'une déchèterie professionnelle,58975.0,aide en numéraire,...,"<p>\n Pour plus d'informations, cliquer sur le...",NaN,NaN,None,2021-11-09T15:42:41+00:00,,False,"[{'name': 'Aide financière', 'id': 1}]",ADEME,8


In [44]:
# Save the handmade DataFrame comming from a automatic matching with a
# Leveinshtein threshold equal to 75.
saveDataFrame(df_new, DATA_FOLDER, "ademe_mt_75.csv")